In [267]:
# Libraries:
import pandas as pd
import numpy as np
from __future__ import division
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')
%pylab inline
import time
import joblib

Populating the interactive namespace from numpy and matplotlib


In [74]:
# Config:
# In order to display all the columns:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

### FUNCTIONS

In [3]:
# Functions to put data in the correct format for estimation:
def get_set_tags(dataframe):
    # Use this function to get the set of tags in the document:
    user_tags=dataframe.usertag
    list_tags=[]
    for i in user_tags:
        for j in i.split(','):
            list_tags.append(j)
    set_tags=list(set(list_tags))
    return set_tags
# removing the null tags:
def remove_objects(list_values,obj="null"):
    return [x for x in list_values if x != obj]

# Functions to separate the usertag columns:
def create_tag_index_dictio(training_df,column="usertag",split_char=","):
    # Creates a dictionary with key: tag and value: list of rows
    tag_series=training_df[column].apply(lambda x:x.split(split_char))
    k=0
    tag_index={}
    for tags_line,index_tag in zip(tag_series,tag_series.index):
        k+=1
        for tag in tags_line:
            if tag != "null":
                if tag not in tag_index.keys():
                    tag_index[tag]=[index_tag]
                else:
                    tag_index[tag].append(index_tag)
        if k%1000000==0:
            print "%d completed"%(k)
    return tag_index

def get_tagsDF(dictio_tag_index,training_set):
    # Create the dictionary with key:usertag, value: list with 1 tha impression correspond to this tag 
    N=training_set.shape[0]
    dictio_tags_values={}
    for user_tag in dictio_tag_index.keys():
        sparce_array=[0]*N
        for i in dictio_tag_index[user_tag]:
            sparce_array[i]=1
        dictio_tags_values[user_tag]=sparce_array
    print "%d users completed, creating dataframe"%(len(dictio_tags_values.keys()))
    return pd.DataFrame(dictio_tags_values)

def normalize(dataframe,column="slotprice"):
    dataframe[column]=(dataframe[column]-dataframe[column].mean())/dataframe[column].std()
    



In [166]:
# Constants:
# According to the iPINyou paper:
columns_to_dummy=["weekday","hour","region","city","adexchange","slotwidth","slotheight","slotvisibility",
                  "slotformat"]
# For train and validation:
columns_to_drop=["bidid","logtype","userid","urlid","url","bidprice","payprice","keypage","creative","slotid"]
# For test:
#columns_to_drop=["bidid","logtype","userid","urlid","url","keypage","creative","domain","IP","slotid"]

### Importing datasets:

In [5]:
# Training set:
start_time=time.time()
filepath="../dataset/train.csv"  ### -----> put the path where is your file
filepath_val="../dataset/validation.csv"
#filepath="../dataset/test.csv"
data=pd.read_csv(filepath)
print "data imported, num rows: %d, num cols: %d"%(data.shape[0],data.shape[1])


data imported, num rows: 2697738, num cols: 26


In [ ]:
# Validation set:
filepath_val="../dataset/validation.csv"
data_val=pd.read_csv(filepath_val)

In [94]:
# Validation set:
filepath_test="../dataset/test.csv"
data_test=pd.read_csv(filepath_test)

In [167]:
# Sampling:
data_0=data[data.click==0]
data_1=data[data.click==1]

In [168]:
# Sampling:
N=data_0.shape[0]
dim=data_0.shape[1]
n_samples= int(N*0.001)
#data_0=data_0.loc[np.sample(data_0.index, n_samples)]
data_0=data_0.loc[np.random.choice(data_0.index,n_samples , replace=False)]


In [169]:
# Get browser, OS:
data_0['browser'], data_0['os'] = data_0['useragent'].str.split('_', 1).str
data_0.drop(["useragent"],axis=1,inplace=True)

In [170]:
# expand df.tags into its own dataframe
tags = data_0['usertag'].apply(pd.Series)

# rename each variable is tags
#tags = tags.rename(columns = lambda x : 'tag_' + str(x))

In [171]:
data_0.drop(columns_to_drop, axis=1, inplace=True)


In [187]:
data_0["lala"]=data.click

In [464]:
from collections import defaultdict
import csv
def get_std_slotprice(path,column="slotprice"):
    df = pd.read_csv(path, skipinitialspace=True, usecols=[column])
    std_value=int(df.slotprice.values.std())
    del(df)
    return std_value

def update_usertag_os_browser(instance,row,col_usertag=25,col_os_brow=6):
    usertags = row[col_usertag].split(',')
    temp_dict = {}
    for tag in usertags:
        temp_dict["tag " + tag] = True
    instance.update(temp_dict)
    # add OS and browser:
    op_sys, browser = row[col_os_brow].split('_')
    instance.update({op_sys: True, browser: True})
    return instance

def process_event(row,STD_SLOTPRICE,data_type="Training"):
    # Initilize instance:
    if data_type=="Training":
        instance = {'weekday': row[1], 'hour': row[2], 'region': row[8], \
                    'city': row[9], 'adexchange': row[10], 'slotwidth': row[15], 'slotheight': row[16], \
                    'slotvisibility': row[17], 'slotformat': row[18], 'slotprice': float(row[19]) / STD_SLOTPRICE, \
                    'advertiser': row[24]}
    elif data_type=="Validation":
        instance = {'weekday': row[1], 'hour': row[2], 'region': row[8], \
                    'city': row[9], 'adexchange': row[10], 'slotwidth': row[15], 'slotheight': row[16], \
                    'slotvisibility': row[17],'payprice':int(row[22]), 'slotformat': row[18], 'slotprice': float(row[19]) / STD_SLOTPRICE, \
                    'advertiser': row[24]}
    else: 
        print "must be training of validation"
        return 
    # Add usertags:
    usertags = row[25].split(',')
    temp_dict = {}
    for tag in usertags:
        temp_dict["tag " + tag] = True
    instance.update(temp_dict)
    # add OS and browser:
    op_sys, browser = row[6].split('_')
    instance.update({op_sys: True, browser: True})
    return instance

In [465]:
def load_data(filepath,data_type="Training"):
    data = list()
    labels = list()
    # std of slotprice for nomalization
    STD_SLOTPRICE = get_std_slotprice(filepath,column="slotprice")
    print "std stored"
    with open(filepath, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        # pass header:
        next(reader)
        # Iterate:
        for row in reader:
            instance=process_event(row,STD_SLOTPRICE,data_type)
            data.append(instance)
            labels.append(int(row[0]))
    print "data and labels loaded"
    return data,labels

In [475]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [509]:
# Loading training data:
new_data=load_data(filepath)
training_events, labels = new_data[0],new_data[1]
del(new_data)
# taking first 50'000:
training_events=training_events[0:100000]
training_labels=labels[0:100000]

std stored
data and labels loaded


In [510]:
# Features: Training
X_train=pd.DataFrame.from_dict(training_events)
X_train=pd.get_dummies(X_train)

In [511]:
# Labels: Training:
Y_train=pd.DataFrame(training_labels,columns=["click"])

In [512]:
# Validation:
val_events,val_labels= load_data(filepath_val)
X_val=pd.DataFrame(val_events)
X_val=pd.get_dummies(X_val)

std stored
data and labels loaded


In [513]:
# Labels Validation
Y_val=pd.DataFrame(val_labels,columns=["click"])

In [522]:
model= LogisticRegression(class_weight="balanced")
model.fit(np.array(X_train),np.array(Y_train))

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [523]:
# Error on the training set:
y_pred_train=model.predict(X_train)
y_true=Y_train.values.reshape(1,len(Y_train))[0]
roc_auc_score(y_pred_train,y_true)

0.50948922211808811

In [524]:
sum(y_pred_train)

4268

In [525]:
sum(y_true)

81

In [173]:
training_set=pd.get_dummies(data_0,columns=columns_to_dummy)

In [174]:
training_set.head()

,click,IP,domain,slotprice,advertiser,usertag,browser,os,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,region_0,region_1,region_2,region_3,region_15,region_27,region_40,region_55,region_65,region_79,region_80,...,city_397,city_398,city_399,adexchange_1,adexchange_2,adexchange_3,adexchange_4,adexchange_null,slotwidth_120,slotwidth_160,slotwidth_180,slotwidth_200,slotwidth_250,slotwidth_300,slotwidth_320,slotwidth_336,slotwidth_360,slotwidth_468,slotwidth_620,slotwidth_640,slotwidth_728,slotwidth_910,slotwidth_950,slotwidth_960,slotwidth_980,slotwidth_1000,slotheight_50,slotheight_60,slotheight_90,slotheight_100,slotheight_200,slotheight_240,slotheight_250,slotheight_280,slotheight_300,slotheight_600,slotvisibility_0,slotvisibility_1,slotvisibility_2,slotvisibility_255,slotvisibility_FirstView,slotvisibility_FourthView,slotvisibility_Na,slotvisibility_OtherView,slotvisibility_SecondView,slotvisibility_ThirdView,slotformat_0,slotformat_1,slotformat_5,slotformat_Na
1916959,0,175.31.48.*,31xSTvprdN1RFt,50,1458,"13403,10006,10059,10063,10111",windows,chrome,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
2179643,0,219.137.249.*,null,250,3386,null,windows,chrome,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0
544446,0,124.116.0.*,5F1RQS9rg5scFsf,50,1458,"10063,10031,10110",windows,chrome,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
2349652,0,117.63.142.*,DSmreF9aBTN7gqKbuKz,4,3476,"10063,10006,10110",windows,ie,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1672540,0,113.6.146.*,3FKElpuEMusyJqKbuKz,50,1458,null,windows,ie,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


In [175]:
# Getting the set of tags:
list_tags=get_set_tags(training_set)
list_tags=remove_objects(list_tags)

# creating dictionaries:
dictio_tag_index=create_tag_index_dictio(training_set)

# creating new dataframe:
N = training_set.shape[0]
tags_valuesDF=get_tagsDF(dictio_tag_index,training_set)

IndexError: list assignment index out of range

In [139]:
data[data.click==1][value].value_counts()[0:10]

0             822
1             381
2             347
FirstView     299
Na            102
OtherView      48
SecondView     17
ThirdView       9
255             7
FourthView      2
Name: slotvisibility, dtype: int64

In [140]:
data[data.click==0][value].value_counts()[0:10]

0             1115365
2              707197
1              300270
Na             191208
FirstView      155958
OtherView      129912
SecondView      34160
ThirdView       20723
255             20049
FourthView      19728
Name: slotvisibility, dtype: int64

In [141]:
data_val[data_val.click==1][value].value_counts()[0:10]

0             79
2             48
1             39
FirstView     35
Na            10
OtherView      9
SecondView     3
255            2
FourthView     1
Name: slotvisibility, dtype: int64

In [142]:
data_val[data_val.click==0][value].value_counts()[0:10]

0             123767
2              78598
1              33521
Na             21124
FirstView      17237
OtherView      14474
SecondView      3816
ThirdView       2416
255             2240
FourthView      2214
Name: slotvisibility, dtype: int64

In [126]:
data_test[value].value_counts()[0:10]

1458    59789
3386    55559
3427    50149
3476    38630
3358    34038
2821    25628
2259    16276
2261    13499
2997     6181
Name: advertiser, dtype: int64

In [150]:
x=pd.get_dummies(data_0.apply(pd.Series).stack()).sum(level=0)


In [ ]:
len(set(data.domain))

In [ ]:
x=data[0:100].iterrows()

In [ ]:
next(x)

In [ ]:
instance = {'weekday': row[1], 'hour': row[2], 'region': row[8], \
            'city': row[9], 'adexchange': row[10], 'slotwidth': row[15], 'slotheight': row[16], \
            'slotvisibility': row[17], 'slotformat': row[18], 'slotprice': float(row[19]) / STD_SLOTPRICE, \
            'advertiser': row[24]}

In [ ]:
# validation set:
#filepath_val="../dataset/validation.csv"
#data_val=pd.read_csv(filepath_val)

#validation_set=data_val[data_val["bidprice"]>data_val["payprice"]]
#validation_set=validation_set.reset_index(drop=True)
#print "validation set: %d rows"%(validation_set.shape[0])

In [ ]:
def drop_inconsistent(data):
    if "bidprice" in data.columns:
        data=data[data["bidprice"]>data["payprice"]]
        return data.reset_index(drop=True)
    


In [ ]:
# Droping inconsistent data:
training_set=data[data["bidprice"]>data["payprice"]]
training_set=training_set.reset_index(drop=True)
print "training set: %d rows"%(training_set.shape[0])

# Droping columns that we don't use:
training_set.drop(columns_to_drop, axis=1, inplace=True)
print "remaining cols: %d" % (training_set.shape[1])

In [ ]:
# Getting the set of tags:
list_tags=get_set_tags(training_set)
list_tags=remove_objects(list_tags)

# creating dictionaries:
dictio_tag_index=create_tag_index_dictio(training_set)
dictio_os_browser=create_tag_index_dictio(training_set,"useragent","_")

# creating new dataframe:
N = training_set.shape[0]
tags_valuesDF=get_tagsDF(dictio_tag_index,training_set)
os_browserDF=get_tagsDF(dictio_os_browser,training_set)

In [ ]:
# Creating categorical Variables:
training_set=pd.get_dummies(training_set,columns=columns_to_dummy)
normalize(training_set)

In [ ]:
# Concatenating
clean_data = pd.concat([training_set, tags_valuesDF,os_browserDF], axis=1)
clean_data.drop(["useragent","usertag"],axis=1,inplace=True)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
del(data)
del(training_set)
clean_data.head()

In [ ]:
import copy
clean_data1=copy.deepcopy(clean_data)

In [ ]:
clean_data1.head()

In [ ]:
clean_data.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
clean_data.shape

In [ ]:
x_0=clean_data[clean_data.click==0]
x_1=clean_data[clean_data.click==1]

In [ ]:
x_0.shape

In [ ]:
x_1.shape

In [ ]:
X_train=np.array(clean_data)
X_train=X_train[:,1:]

In [ ]:
X_train.shape

In [ ]:
from sklearn.decomposition import PCA
X = np.array(x_1)
X = X[:,1:]
pca = PCA(n_components=200)
pca.fit(X)


In [ ]:
plt.plot(pca.explained_variance_ratio_)

In [ ]:
pca_x1=pca.fit_transform(x_1)

In [ ]:
pca_x1.shape

In [ ]:
# Saving file:
data_save = np.array(clean_data)

In [ ]:
joblib.dump(data_save, './data_for_CTR/full_data_val.pkl', compress=True)

In [ ]:
# Saving one file:
pd14=clean_data[clean_data.advertiser==1458].drop("advertiser",axis=1)
train14=np.array(pd14)
#np.save("data_train_1458",data_train_1458)

In [ ]:
train14=np.array(pd14)

In [ ]:
clean_train
joblib.dump(clean_train, './data_for_CTR/full_data.pkl', compress=True)

In [ ]:
#joblib.dump(train14, './data_for_CTR/val_data_1458.pkl', compress=True)

In [ ]:
#back14=joblib.load('test_dump.pkl')

In [ ]:
#pd14.to_csv("pd_14_val.csv",index=False)

In [ ]:
# Author: Matt Terry <matt.terry@gmail.com>
#
# License: BSD 3 clause
from __future__ import print_function

import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


In [ ]:


class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text),
                 'num_sentences': text.count('.')}
                for text in posts]


class SubjectBodyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        features = np.recarray(shape=(len(posts),),
                               dtype=[('subject', object), ('body', object)])
        for i, text in enumerate(posts):
            headers, _, bod = text.partition('\n\n')
            bod = strip_newsgroup_footer(bod)
            bod = strip_newsgroup_quoting(bod)
            features['body'][i] = bod

            prefix = 'Subject:'
            sub = ''
            for line in headers.split('\n'):
                if line.startswith(prefix):
                    sub = line[len(prefix):]
                    break
            features['subject'][i] = sub

        return features


pipeline = Pipeline([
    # Extract the subject & body
    ('subjectbody', SubjectBodyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for pulling features from the post's subject line
            ('subject', Pipeline([
                ('selector', ItemSelector(key='subject')),
                ('tfidf', TfidfVectorizer(min_df=50)),
            ])),

            # Pipeline for standard bag-of-words model for body
            ('body_bow', Pipeline([
                ('selector', ItemSelector(key='body')),
                ('tfidf', TfidfVectorizer()),
                ('best', TruncatedSVD(n_components=50)),
            ])),

            # Pipeline for pulling ad hoc features from post's body
            ('body_stats', Pipeline([
                ('selector', ItemSelector(key='body')),
                ('stats', TextStats()),  # returns a list of dicts
                ('vect', DictVectorizer()),  # list of dicts -> feature matrix
            ])),

        ],

        # weight components in FeatureUnion
        transformer_weights={
            'subject': 0.8,
            'body_bow': 0.5,
            'body_stats': 1.0,
        },
    )),

    # Use a SVC classifier on the combined features
    ('svc', SVC(kernel='linear')),
])

# limit the list of categories to make running this example faster.
categories = ['alt.atheism', 'talk.religion.misc']
train = fetch_20newsgroups(random_state=1,
                           subset='train',
                           categories=categories,
                           )
test = fetch_20newsgroups(random_state=1,
                          subset='test',
                          categories=categories,
                          )

pipeline.fit(train.data, train.target)
y = pipeline.predict(test.data)
print(classification_report(y, test.target))